In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
! pip install pandas dash


In [ ]:
#! wget "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv"

In [ ]:
#!wget "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/spacex_dash_app.py"


In [ ]:
# Import required libraries
import pandas as pd
import dash
#import dash_html_components as html
from dash import html
#import dash_core_components as dcc
from dash import dcc
from dash.dependencies import Input, Output
import plotly.express as px

In [ ]:
# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("../input/spacex-launch-dash/spacex_launch_dash.csv")
spacex_df.head()

In [ ]:
launch_site_df = spacex_df[['Launch Site','class']]
#launch_site_df = launch_site_df.groupby('Launch Site').sum()
launch_site_df.head()


In [ ]:
#make a pie chart for all site with successful rate
#import plotly.express as px
# This dataframe has 244 lines, but 4 distinct values for `day`
#df = px.data.tips()
fig = px.pie(launch_site_df, values='class', names='Launch Site')
fig.show()

In [ ]:
# show success rate over failure rate for specifical  launch site
launch_site ='KSC LC-39A'
#sub dataframe for choosen launch_site
sub_launch_site_df = spacex_df[spacex_df['Launch Site']==launch_site]
sub_launch_site_df = sub_launch_site_df[['class','Flight Number']]
sub_launch_site_df.head()

In [ ]:
import numpy as np
#add a new categorize column for sub_launch_site_df
#create a list of condition
conditions = [(sub_launch_site_df['class']==0),sub_launch_site_df['class']==1]
#create a list of value we want to assign for each condition
values=['failure','success']
#create a new column and use np.select to assign values to it
#bar_data = data.groupby(['DestState'])['Flights'].sum().reset_index()
sub_launch_site_df['fail_or_success']=np.select(conditions, values)
pie_data = sub_launch_site_df.groupby(['fail_or_success'])['Flight Number'].count().reset_index()
               


In [ ]:
#make a pie chart
fig1 = px.pie(pie_data,names ='fail_or_success',values='Flight Number')
fig1.show()

In [ ]:
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

# Create a dash application
app = dash.Dash(__name__)

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                # dcc.Dropdown(id='site-dropdown',...)
                                """
                                dcc.Dropdown(id='id',options=[{'label': 'All Sites', 'value': 'ALL'},{'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
                                                             {'label':'CCAFS SLC-40','value':'CCAFS SLC-40'},{'label':'KSC LC-39A','value':'KSC LC-39A'},
                                                             {'label':'VAFB SLC-4E','value':'VAFB SLC-4E'}],
                                             value='ALL',placeholder="place holder here",searchable=True),
                                html.Br(),
                                """
                                #use smarter way to build dropdown
                                dcc.Dropdown(space_df['Launch Site'].unique(),
                                             id = 'site-dropdown',
                                            value='ALL', placeholder = "place holder here", searchable = True),
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                #dcc.RangeSlider(id='payload-slider',...)

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

In [ ]:
# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback(
    Output('success-pie-chart','figure'),
    Input('site-dropdown','value'))
def launch_site_piechart(launch_site):
    df =spacex_df[spacex_df['Launch Site']==launch_site]
    
    #group by the data by class and count the value 
    pie_df = df.groupby('class')['Flight Number'].count_values().reset_index()
    
    fig = px.pie(pie_data, names = 'class', values = 'Flight Number')
    fig.show()
    return fig
    
# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output


# Run the app
#use exit to quit
#if __name__ == '__main__':
    app.run_server()